In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyecharts.options as opt
import warnings
from pyecharts.globals import CurrentConfig, NotebookType
from pypalettes import load_cmap, get_hex
from pyecharts.charts import Pie

# 忽略警告信息
warnings.filterwarnings('ignore')

# Matplotlib配置
plt.rcParams["font.family"] = 'kaiti'  # 设置中文字体为楷体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# Pyecharts笔记本配置
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK  # 设置为Jupyter Notebook环境


In [ ]:
df1=pd.read_excel(r"E:\存放桌面\数学建模资料汇总数学建模BOOM\2025暑期任务-2\C题\附件1.xlsx",index_col=0,sheet_name="乡村的现有耕地").drop('说明 ',axis=1)
#print("列名列表：", df1.columns.tolist())  # tolist() 转换为普通列表格式
#牛逼，有个空格

df2=pd.read_excel(r"E:\存放桌面\数学建模资料汇总数学建模BOOM\2025暑期任务-2\C题\附件1.xlsx",index_col=1,sheet_name=1).drop('说明',axis=1)
#print("列名列表：", df2.columns.tolist())  # tolist() 转换为普通列表格式
df1


In [ ]:
#print("行名称列表：", df2.index.tolist())

#df2.drop(['43','46'],axis=0)  # 传入列表
df2.iloc[42:45]#索引是最左边的文字，不是第几行；要定位可以用iloc 前边是行，后边是列，注意下标从0开始
print(df2.iloc[42:45])

In [ ]:
df2.iloc[:, 1:4]#原来最后的说明那一列已经被删除了

In [ ]:
island=df1.loc['A1':'D8']
island


In [ ]:
group_type=island.groupby('地块类型').sum().reset_index()
lst=group_type.values.tolist()
lst
"""这两行代码是使用Pandas库对数据进行分组统计和格式转换的操作：

group_type = island.groupby('地块类型').sum().reset_index()

对island这个DataFrame按'地块类型'列进行分组
对每个分组内的数值型列进行求和运算
通过reset_index()将分组结果重新设置为标准DataFrame格式（将分组键'地块类型'从索引转为普通列）
lst = group_type.values.tolist()

将DataFrame格式的group_type转换为NumPy数组（通过.values）
再将NumPy数组转换为Python列表格式
最终lst会是一个嵌套列表，每个子列表代表一行数据，包含'地块类型'和对应的求和结果"""

In [ ]:
outter = list()# 创建一个空列表，用于存储最终结果
for i in lst:  # i 是 lst 中的每个子元素（如 ['类型A']）
    name = i[0]  # 提取子元素中的第一个值作为名称（如 '类型A'）
    # 从 island 中筛选出“地块类型”等于 name 的行，再提取这些行的“地块面积/亩”列
    select = island.query('地块类型 == @name')['地块面积/亩']
    num = 1  # 初始化计数器，用于给同一类型的地块编号
    for j in select.values.tolist():  # 遍历筛选出的面积数据（转换为普通列表）
    # 拼接成 (名称_编号, 面积) 的元组，添加到 outter 列表
        outter.append((f'{name}_{num}', j))
        num += 1  # 计数器自增，确保每个地块编号唯一
outter

In [ ]:
from pyecharts import options as opt
from pyecharts.charts import Pie
from pypalettes import load_cmap

# 加载颜色映射（连续型的"pastel"色系）
cmap = load_cmap('pastel', cmap_type='continuous')

# 创建饼图实例
pie = Pie()

# 添加内层饼图（地块类型）
pie.add(
    series_name="地块类型",  # 系列名称（用于 tooltip 等）
    data_pair=lst,  # 数据对（格式：[(类型1, 数值1), (类型2, 数值2), ...]）
    radius=[0, '50%'],  # 内层饼图半径（0到50%处）
    label_opts=opt.LabelOpts(  # 标签配置
        position='inside',  # 标签在饼图内部
        is_show=True,  # 显示标签
        formatter='{b}\n{d}%',  # 标签格式：名称 + 百分比（{b}是名称，{d}是百分比）
        overflow='none',
        rich={  # 自定义样式
            "b": {"fontSize": 16, "fontFamily": "KaiTi"}  # 名称字体：16号楷体
        }
    ),
    percent_precision=0,  # 百分比保留0位小数
    is_avoid_label_overlap=False  # 不避免标签重叠
)

# 设置饼图颜色（使用颜色映射的前4种颜色）
pie.set_colors(cmap.colors[:4])

# 添加外层饼图（地块子类）
pie.add(
    series_name="地块子类",
    data_pair=outter,  # 数据对（格式同内层，如[('耕地_1', 数值), ...]）
    radius=['60%', '75%'],  # 外层饼图半径（60%到75%处）
    label_opts=opt.LabelOpts(  # 标签配置
        position='outside',  # 标签在饼图外部
        is_show=True,
        formatter="{b}: {d}%",  # 标签格式：名称: 百分比
        background_color="#eee",  # 标签背景色
        border_color="#aaa",  # 标签边框色
        border_width=1,  # 边框宽度
        border_radius=4  # 边框圆角
    ),
    is_legend_hover_link=False  # 关闭图例 hover 联动
)

# 设置全局配置
pie.set_global_opts(
    legend_opts=opt.LegendOpts(is_show=False),  # 不显示图例
    title_opts=opt.TitleOpts(  # 标题配置
        title="耕地结构",  # 主标题
        subtitle="露天耕地结构",  # 副标题
        pos_left="center"  # 标题居中
    )
)

# 设置系列配置（提示框）
pie.set_series_opts(
    tooltip_opts=opt.TooltipOpts(  # 鼠标悬停提示框
        trigger="item",  # 按数据项触发
        formatter="{a} <br/>{b}: {c}({d}%)"  # 格式：系列名 + 名称: 数值(百分比)
    )
)

# 在 Jupyter Notebook 中渲染图表
pie.render_notebook()

In [ ]:
inner = pd.DataFrame(lst).set_index(0)
print(inner)
inner = inner / inner.sum()
inner

In [ ]:
out = pd.DataFrame(outter).set_index(0)
out /= out.sum()

outer_color_index=list([i + 5 for i in list(range(6))])
outer_color_index.extend([i + 20 for i in list(range(6))])
outer_color_index.extend([i + 40 for i in list(range(14))])
outer_color_index.extend([i + 80 for i in list(range(8))])
inner_camp =cmap([0,15,35,75])
outter_cmap=cmap(outer_color_index)

plt.figure(figsize=(8,8))
plt.pie(inner[1],
    labels=inner.index,
    radius=0.6,
    autopct="%1.1f%%",
    colors=inner_camp,
    labeldistance=0.3,
    wedgeprops=dict(width=0.6,edgecolor='white'))
plt.legend()
plt.pie(out[1],
    radius=1,
    colors=outter_cmap,
    wedgeprops=dict(width=0.4,edgecolor='white'))
plt.tight_layout()


In [ ]:
notland=df1.drop(island.index)
notland

In [ ]:
group2=notland.groupby('地块类型').sum()
group2/=group2.sum()
group2

In [ ]:
group2_out=notland.set_index('地块类型')
group2_out/=group2_out.sum()
group2_out

In [ ]:
import matplotlib.pyplot as plt

# 生成外层颜色的索引列表
outer_color_index = [i + 5 for i in range(16)]  # 生成 5-20 的索引（共16个）
outer_color_index.extend([i + 80 for i in range(14)])  # 追加 80-83 的索引（共4个）

# 获取内圈和外圈的颜色（假设 cmap 是已加载的颜色映射）
inner_cmap = cmap([5, 75])  # 从颜色映射中取索引为5和75的颜色（内圈用）
outer_cmap = cmap(outer_color_index)  # 用生成的索引取外圈颜色

# 创建画布
plt.figure(figsize=(8, 8))  # 设置画布大小为8x8

# 绘制内圈饼图
plt.pie(
    group2['地块面积/亩'],  # 内圈数据（地块面积）
    labels=group2.index,   # 内圈标签（地块类型）
    radius=0.6,            # 内圈半径（相对值）
    autopct="%1.1f%%",     # 显示百分比（保留1位小数）
    colors=inner_cmap,     # 内圈颜色
    labeldistance=0.3,     # 标签位置（距离圆心的比例）
    wedgeprops=dict(width=0.6, edgecolor='white')  # 内圈样式（宽度、边框白色）
)

# 添加图例
plt.legend()

# 绘制外圈饼图
plt.pie(
    group2_out['地块面积/亩'],  # 外圈数据
    radius=1,                   # 外圈半径（相对值，大于内圈）
    colors=outer_cmap,          # 外圈颜色
    wedgeprops=dict(width=0.4, edgecolor='white')  # 外圈样式（宽度、边框白色）
)

# 调整布局，避免元素重叠
plt.tight_layout()

# 显示图像
plt.show()

In [ ]:
df3=pd.read_excel(r"E:\存放桌面\数学建模资料汇总数学建模BOOM\2025暑期任务-2\C题\附件2.xlsx").ffill().set_index('种植地块',drop=True)
df3
"""
填充缺失值
.ffill()


这是fillna(method='ffill')的简写形式
作用是 "向前填充"：用前一行的非缺失值填充当前行的缺失值
常用于处理连续数据中出现的空值，保持数据的连续性


设置索引
.set_index('种植地块', drop=True)


将 "种植地块" 这一列设置为 DataFrame 的行索引
drop=True表示将 "种植地块" 列从原数据中删除（不再作为普通列存在）
设置后可以通过 "种植地块" 的名称快速定位数据
"""

In [ ]:
df4 = pd.read_excel(r"E:\存放桌面\数学建模资料汇总数学建模BOOM\2025暑期任务-2\C题\附件2.xlsx",index_col=0,sheet_name='2023年统计的相关数据').dropna(axis=0,how='any')
# 读取文件：从指定路径的 Excel 中读取 “2023 年统计的相关数据” 工作表。
# 设置索引：index_col=0 表示用表格的第 1 列（默认从 0 开始计数）作为行索引。
# 删除缺失值：dropna(axis=0, how='any') 表示删除所有包含缺失值（NaN）的行（axis=0 指行，how='any' 指只要有一个缺失值就删除）。                                                                                                                    
temp = df4.query('地块类型 =="普通大棚 " and 种植季次 =="第一季"')
# 用 query() 方法筛选出满足两个条件的行：
# 地块类型 为 普通大棚 （注意原字符串末尾有空格）
# 种植季次 为 第一季
# 筛选结果存入临时变量 temp。
temp['地块类型']='智慧大棚'
# 将 temp 中所有行的 地块类型 列的值从 普通大棚 改为 智慧大棚。
# 相当于复制了 “普通大棚 第一季” 的数据，并将其标记为 “智慧大棚”。
df4 = pd.concat([df4,temp],axis=0)
df4 = df4.reset_index(drop=True)
# 合并数据：pd.concat(..., axis=0) 将原 df4 和修改后的 temp 纵向拼接（增加行数），相当于在原数据后追加了修改后的副本。
# 重置索引：reset_index(drop=True) 清除合并后可能重复的原索引，重新生成从 0 开始的整数索引。
df4

In [ ]:
df3['地块类型']= np.nan
#给 df3 新增一列 '地块类型'
#用 np.nan（pandas 中的缺失值标记）初始化这一列，此时所有行的 '地块类型' 都是缺失值
# 在 pandas 的 query() 方法中，@ 是一个特殊语法，用于引用当前环境中的变量（即查询字符串外部定义的变量）。
# 它的作用是告诉 pandas：@ 后面的名称不是 DataFrame 中的列名，而是一个外部变量，需要从当前代码的上下文（环境）中获取它的值。
for i in df1.index:  # 遍历 df1 的所有行索引
    # 筛选出 df3 中索引等于 i 的行（理论上应该是单行）
    select = df3.query('@df3.index == @i')
    
    # 将 df1 中索引为 i 的 '地块类型' 值，赋给 df3 中相同索引 i 的 '地块类型'
    df3.loc[i, '地块类型'] = df1.loc[i, '地块类型']
                              
df3

In [ ]:
# 复制df3到union（保留df3原始数据，在副本上操作）
union = df3.copy()

# 给union新增4列，并初始化为缺失值（NaN）
union[['种植季次', '亩产量', '种植成本', '销售单价']] = np.nan

# 遍历df3的每一行（idx是行索引，row是每行数据）
for idx, row in df3.iterrows():
    # 从df4中筛选出符合条件的行：
    # 条件1：df4的“作物编号”等于当前行（row）的“作物编号”
    # 条件2：df4的“地块类型”等于当前行（row）的“地块类型”
    filtered_df4 = df4[(df4['作物编号'] == row['作物编号']) & (df4['地块类型'] == row['地块类型'])]
    
    # 将筛选出的df4中“种植季次”及之后的列的第一行数据，填充到union的对应行
    union.loc[idx, ['种植季次', '亩产量', '种植成本', '销售单价']] = filtered_df4.loc[:, '种植季次':].values[0]

union

In [ ]:
union["作物类型"]=union["作物类型"].apply(lambda x:x.split('（')[0].strip())
"""这行代码的作用是对 union 数据框中 ["作物类型"] 列的内容进行清洗和格式化，具体来说是提取括号前的文本并去除前后空格。
代码拆解
union["作物类型"]：选中 union 中的 作物类型 列。
apply(lambda x: ...)：对该列的每个值（x）应用匿名函数处理。
x.split('（')[0]：
split('（')：按中文左括号 （ 分割字符串（注意与英文括号 ( 的区别，中文括号是全角字符）。
例如：
若 x 是 "小麦（冬麦）"，分割后得到 ["小麦", "冬麦）"]
若 x 是 "玉米（春播）"，分割后得到 ["玉米", "春播）"]
[0]：取分割后列表的第一个元素，即中文括号前的核心文本。
.strip()：去除提取后文本前后的空白字符（如空格、换行等）。"""
union

In [ ]:
union['产量']=union['种植面积/亩']*union['亩产量']
# 在 union 数据框中新增一列 '产量'，并通过计算得出该列的值
#在 pandas 中，可以直接给 DataFrame 增加新列，这是非常常用的操作。你只需要通过 df['新列名'] = 值 的方式赋值，pandas 会自动为 DataFrame 新增一列并填充数据。
union

In [ ]:
# 按“作物名称”分组，聚合“作物类型”（取众数）和“产量”（求和），然后排序
stats1 = union.groupby('作物名称').agg(
    作物类型=pd.NamedAgg(column='作物类型', aggfunc=lambda x: x.mode().iloc[0]),  # 取作物类型的众数
    产量=pd.NamedAgg(column='产量', aggfunc='sum')  # 产量求和
).reset_index()  # 重置索引，将分组列“作物名称”从索引变回普通列
stats1 = stats1.sort_values(by=['作物类型','产量'],ascending=[False, True])  # 按“作物类型”和“产量”排序

"""在 pandas 中，使用 sort_values() 函数可以灵活规定排序原则，包括排序依据（按哪些列排序）、排序方向（升序 / 降序）、缺失值处理等。以下是常见的排序原则设置方法：
1. 基本排序：指定列和方向
最常用的方式是通过 by 参数指定排序依据的列，通过 ascending 参数指定升序 / 降序（True 为升序，False 为降序）。
示例：
python
运行
# 按“作物类型”升序（默认），同一类型内按“产量”降序
stats1 = stats1.sort_values(by=['作物类型', '产量'], ascending=[True, False])
by=['作物类型', '产量']：先按 “作物类型” 排序，再按 “产量” 排序（优先级从左到右）。
ascending=[True, False]：对应 by 中的列，“作物类型” 升序，“产量” 降序。
2. 单独指定某列的排序方向
如果只需对部分列指定方向，其他列用默认值（升序），可以简化写法：
python
运行
# 按“产量”降序（只指定这一列的方向，其他列默认升序）
stats1 = stats1.sort_values(by='产量', ascending=False)
3. 按自定义顺序排序（非字母 / 数字顺序）
如果需要按自定义的类别顺序排序（例如 “粮食”→“蔬菜”→“食用菌” 的特定顺序），可以先将列转为 Categorical 类型，再排序。
示例：
python
运行
# 定义“作物类型”的自定义排序顺序
custom_order = ['粮食', '蔬菜', '食用菌']

# 将“作物类型”列转为分类类型，并指定顺序
stats1['作物类型'] = pd.Categorical(stats1['作物类型'], categories=custom_order, ordered=True)

# 按自定义顺序排序
stats1 = stats1.sort_values(by='作物类型')
此时排序会严格按照 custom_order 中定义的顺序（粮食→蔬菜→食用菌），而非默认的字母顺序。
4. 处理缺失值
默认情况下，缺失值（NaN）会被排在最后。可以通过 na_position 参数调整：
na_position='last'（默认）：缺失值排在最后。
na_position='first'：缺失值排在最前。
python
运行
# 让缺失值排在最前面
stats1 = stats1.sort_values(by='产量', na_position='first')
5. 按索引排序
如果需要按行索引排序，而非列的值，可以使用 sort_index() 函数：
python
运行
# 按索引升序排序
stats1 = stats1.sort_index(ascending=True)

总结
排序原则的核心参数：
参数	作用	示例
by	指定排序依据的列（列表或单个列）	by=['作物类型', '产量']
ascending	指定升序 / 降序（布尔值或列表）	ascending=[True, False]
na_position	缺失值位置（'last'/'first'）	na_position='first'
通过组合这些参数，可以满足绝大多数排序需求，例如 “先按自定义类型顺序排序，同类型内按产量降序，缺失值放最后” 等复杂逻辑。"""

stats1  # 显示结果

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 创建画布，设置大小为12x12
plt.figure(figsize=(12, 12))

# 绘制横向条形图
sns.barplot(
    data=stats1,           # 数据源
    x="产量",               # x轴为“产量”
    y="作物名称",           # y轴为“作物名称”
    hue="作物类型",         # 按“作物类型”分组显示不同颜色
    orient="h",            # 横向条形图（h表示horizontal）
    palette={              # 自定义颜色映射
        "食用菌": "lightblue",
        "粮食": "gold",
        "蔬菜": "green"
    },
    width=1.1,             # 条形宽度
    gap=0.2                # 组内条形间隙
)

# 设置x轴标签及字体大小
plt.xlabel("产量", fontdict={"fontsize": 16})

# 设置y轴标签及字体大小
plt.ylabel("作物名称", fontdict={"fontsize": 16})

# 调整y轴刻度标签与轴的距离
plt.tick_params(axis='y', pad=5)

# 设置图例字体大小
plt.legend(fontsize=16)

# 自动调整布局，避免元素重叠
plt.tight_layout()

# 显示图形
plt.show()

In [ ]:
mapper=union[['作物名称','作物类型']].set_index('作物名称').to_dict(orient='dict')['作物类型']
"""这行代码的作用是创建一个从 “作物名称” 映射到 “作物类型” 的字典，方便后续通过作物名称快速查询对应的作物类型。
代码拆解步骤：
提取需要的列
union[['作物名称','作物类型']]
从 union 数据框中筛选出 “作物名称” 和 “作物类型” 两列，形成一个只包含这两列的子数据框。
设置索引
.set_index('作物名称')
将 “作物名称” 列设置为子数据框的行索引，此时数据结构变为：
作物名称	作物类型
小麦	粮食
白菜	蔬菜
转换为字典
.to_dict(orient='dict')
将数据框转换为字典，orient='dict' 是默认格式，生成的字典结构为：
python
运行
{'作物类型': {'小麦': '粮食', '白菜': '蔬菜', ...}}  # 外层键是列名，内层键是索引（作物名称）

提取目标映射
['作物类型']
从上述字典中取出 “作物类型” 对应的值，最终得到：
python
运行
{'小麦': '粮食', '白菜': '蔬菜', ...}  # 键是作物名称，值是对应的作物类型

最终结果
mapper 是一个字典，格式为 {作物名称: 作物类型, ...}，例如：
python
运行
{
    '小麦': '粮食',
    '玉米': '粮食',
    '白菜': '蔬菜',
    '香菇': '食用菌'
}

用途
这个字典可以快速实现 “通过作物名称查询作物类型”，例如：
python
运行
print(mapper['小麦'])  # 输出：'粮食'
print(mapper['白菜'])  # 输出：'蔬菜'"""
mapper

In [ ]:
# 给df4新增“作物类型”列，通过作物名称映射得到（使用之前创建的mapper字典）
df4['作物类型'] = df4['作物名称'].map(mapper)

# 筛选出“作物类型为粮食”且“地块类型不是水流地”的行
select = df4.query("作物类型 == '粮食' and 地块类型 != '水浇地'")

# 显示筛选结果
select

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 创建画布，设置尺寸为8x5
plt.figure(figsize=(8, 5))

# 绘制分组条形图
sns.barplot(
    data=select,  # 数据源为筛选后的select数据框
    x='作物名称',  # x轴为作物名称
    y='亩产量/斤',  # y轴为亩产量
    hue='地块类型',  # 按地块类型分组（不同颜色区分）
    palette=sns.color_palette('pastel')  # 使用pastel色系
)

# 显示图例（标识不同颜色对应的地块类型）
plt.legend()

# 自动调整布局，避免元素重叠
plt.tight_layout()

# 显示图形
plt.show()

In [ ]:
# 筛选出作物类型为“蔬菜”的行
select = df4.query('作物类型 == "蔬菜"')

# 筛选出地块类型为“智慧大棚”的行
modern = df4.query('地块类型 == "智慧大棚"')

# 将“智慧大棚”的地块类型修改为“智慧大棚+种植季次”（如“智慧大棚第一季”）
modern['地块类型'] = modern['地块类型'] + modern['种植季次']

# 用modern的数据更新select中索引相同的行（覆盖相同索引的记录）
select.update(modern)

# 显示更新后的结果
select

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 创建画布，设置大小为10x6
plt.figure(figsize=(10, 6))

# 绘制分组条形图
sns.barplot(
    data=select,  # 数据源为筛选后的蔬菜数据
    x='作物名称',  # x轴为作物名称
    y='亩产量/斤',  # y轴为亩产量（单位：斤）
    hue='地块类型',  # 按地块类型分组（不同颜色区分）
    palette=sns.color_palette('pastel')  # 使用柔和的pastel色系
)

# 显示图例（标识不同颜色对应的地块类型）
plt.legend()

# 自动调整布局，避免元素重叠
plt.tight_layout()

# 显示图形
plt.show()


In [ ]:
select= df4.query('作物类型=="蔬菜"')
modern = df4.query('地块类型=="智慧大棚"')
modern['地块类型']= modern['地块类型']+ modern['种植季次']
select.update(modern)
select

In [ ]:
# 计算平均销售单价：假设销售单价格式为"x-y"（如"2.5-3.5"），取区间平均值
df4['平均销售单价'] = df4['销售单价/(元/斤)'].str.split('-').apply(
    lambda x: (float(x[0]) + float(x[1])) / 2  
)

# 计算平均亩利润：单价×亩产量 - 种植成本
df4['平均亩利润'] = df4['平均销售单价'] * df4['亩产量/斤'] - df4['种植成本/(元/亩)']

# 按作物类型和平均亩利润升序排序（默认）
df4 = df4.sort_values(['作物类型', '平均亩利润']) 
"""计算平均销售单价：
假设 '销售单价/(元/斤)' 列的格式是区间字符串（如 "3.0-5.0" 表示单价在 3.0 到 5.0 元 / 斤之间）。
str.split('-') 将字符串按 - 分割为列表（如 "3.0-5.0" 变为 ['3.0', '5.0']）。
lambda x: (float(x[0]) + float(x[1])) / 2 计算区间的平均值（(起始值 + 结束值)/2），得到平均销售单价。
计算平均亩利润：
公式：平均销售单价 × 亩产量/斤 - 种植成本/(元/亩)
含义：每亩的总收入（单价 × 产量）减去每亩的种植成本，得到每亩的利润。
排序：
sort_values(['作物类型', '平均亩利润']) 先按 “作物类型” 分组排序，同一类型内再按 “平均亩利润” 升序排列，便于观察不同作物类型的利润分布。"""

# 显示结果
df4

In [ ]:
# 按作物名称分组，聚合作物类型（取众数）和平均亩利润（取均值）
stats2 = df4.groupby('作物名称').agg(
    作物类型=pd.NamedAgg(column='作物类型', aggfunc=lambda x: x.mode().iloc[0]),  # 作物类型取众数
    平均亩利润=pd.NamedAgg(column='平均亩利润', aggfunc='mean')  # 平均亩利润取均值
).sort_values(['作物类型', '平均亩利润'])  # 按作物类型和平均亩利润排序
stats2 = stats2.reset_index()  # 重置索引，将作物名称从索引转为列
"""分组聚合：
groupby('作物名称')：按作物名称（如 “白菜”“番茄”）分组
作物类型=pd.NamedAgg(...)：对每组的 “作物类型” 取众数（出现次数最多的类型），确保每种作物对应唯一类型
平均亩利润=pd.NamedAgg(...)：对每组的 “平均亩利润” 取均值，得到该作物的整体平均利润水平
排序与索引重置：
sort_values(['作物类型', '平均亩利润'])：先按作物类型分组排序，同类型内按平均亩利润升序排列
reset_index()：将分组依据 “作物名称” 从索引转为普通列，使结果格式更规范"""

"""agg()：聚合函数
agg() 是 pandas 中用于聚合操作的核心函数，作用是对分组后的每组数据应用一个或多个统计函数，生成汇总结果。"""

"""pd.NamedAgg：命名聚合
pd.NamedAgg 是 pandas 提供的一个辅助类，用于更清晰地定义聚合操作的参数，尤其适合在复杂聚合场景中明确指定 “原列名”“聚合函数” 和 “新列名”。"""
# 显示结果
stats2
    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 创建画布，设置大小为12x10
plt.figure(figsize=(12, 10))

# 绘制横向分组条形图
sns.barplot(
    data=stats2,               # 数据源为统计后的stats2
    x="平均亩利润",             # x轴为平均亩利润
    y="作物名称",               # y轴为作物名称
    hue="作物类型",             # 按作物类型分组（不同颜色区分）
    orient="h",                # 横向条形图
    palette={                  # 自定义颜色映射
        "粮食": "gold",
        "蔬菜": "green",
        "食用菌": "lightblue"
    },
    width=1.1,                 # 条形宽度
    gap=0.2                    # 组内条形间隙
)

# 设置x轴标签及字体大小
plt.xlabel("平均亩利润", fontdict={"fontsize": 16})

# 设置y轴标签及字体大小
plt.ylabel("作物名称", fontdict={"fontsize": 16})

# 调整y轴刻度标签与轴的距离
plt.tick_params(axis='y', pad=5)

# 设置图例字体大小
plt.legend(fontsize=16)

# 自动调整布局，避免元素重叠
plt.tight_layout()

# 显示图形
plt.show()
    